<a href="https://colab.research.google.com/github/SaanidhyaM/RAG-QA_Bot/blob/main/RAG_QA_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pinecone-client

import os
import pinecone
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# load the text file containing business data. needs to be provided along with location
with open('data.txt', 'r') as f:
    data = f.read()

tokenizer =AutoTokenizer.from_pretrained('t5-base')
model= AutoModelForSeq2SeqLM.from_pretrained('t5-base')

# initialize the pinecone client with api key
try:
    pinecone.init(api_key='API_KEY')
except Exception as e:
    print(f"Error initializing Pinecone client: {e}")
    exit(1)
index_name='business_qa_index'
try:
    pinecone.create_index(index_name, dimension=128, metric='cosine', spec=ServerlessSpec(cloud="aws", region="us-east-1"))
except Exception as e:
    print(f"Error creating Pinecone index: {e}")
    exit(1)

# add the text data to the index
vectors=[]
for text in data.split('\n'):
    inputs=tokenizer.encode_plus(text,
                                    add_special_tokens=True,
                                    max_length=512,
                                    return_attention_mask=True,
                                    return_tensors='pt')
    outputs=model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
    vector= outputs.last_hidden_state[:, 0, :]
    vectors.append(vector.detach().numpy())

pinecone.add_vectors(index_name, vectors)

# RAG system
def get_context(message, max_tokens=3000, min_score=0.7, get_only_text=True):
    # Get the embeddings of the input message
    inputs= tokenizer.encode_plus(message,
                                    add_special_tokens=True,
                                    max_length=512,
                                    return_attention_mask=True,
                                    return_tensors='pt')
    outputs =model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
    embedding= outputs.last_hidden_state[:, 0, :]
    results =pinecone.query(index_name, embedding, top_k=10)

    # Filter the results based on the minimum score
    filtered_results= [result for result in results if result.score >= min_score]

    # Return the context as a string or a set of ScoredVectors
    if get_only_text:
        context= ' '.join([result.vector for result in filtered_results])
    else:
        context = filtered_results
    return context

def conversation_chain(messages):
    context= ''
    for message in messages:
        context += get_context(message)
    return {'answer': context}

# Example queries
query1 ="What is the company's mission?"
result1= conversation_chain([query1])
print(result1['answer'])

query2= "What are the company's values?"
result2= conversation_chain([query2])
print(result2['answer'])

query3 ="What is the company's history?"
result3= conversation_chain([query3])
print(result3['answer'])